In [2]:
from ortools.linear_solver import pywraplp
import random

# Create the solver
solver = pywraplp.Solver.CreateSolver('SCIP')

if not solver:
    print('SCIP solver is not available.')
    exit()

# Parameters
num_factories = 100
num_warehouses = 200
num_products = 50

# Randomly generate production capacities, demand, and transportation costs
random.seed(42)
production_capacity = {(f, p): random.randint(500, 1500) for f in range(num_factories) for p in range(num_products)}
demand = {(w, p): random.randint(300, 1200) for w in range(num_warehouses) for p in range(num_products)}
transportation_cost = {(f, w, p): random.uniform(1, 10) for f in range(num_factories) for w in range(num_warehouses) for p in range(num_products)}

# Ensure total production capacity meets or exceeds total demand for each product
for p in range(num_products):
    total_production_capacity = sum(production_capacity[f, p] for f in range(num_factories))
    total_demand = sum(demand[w, p] for w in range(num_warehouses))
    if total_production_capacity < total_demand:
        scale_factor = total_demand / total_production_capacity
        for f in range(num_factories):
            production_capacity[f, p] = int(production_capacity[f, p] * scale_factor) + 1

# Decision variables
shipping = {}
for f in range(num_factories):
    for w in range(num_warehouses):
        for p in range(num_products):
            shipping[f, w, p] = solver.IntVar(0, solver.infinity(), f'shipping_{f}_{w}_{p}')

# Objective: Minimize total transportation cost
objective = solver.Objective()
for f in range(num_factories):
    for w in range(num_warehouses):
        for p in range(num_products):
            objective.SetCoefficient(shipping[f, w, p], transportation_cost[f, w, p])
objective.SetMinimization()

# Constraints

# Production capacity constraint
for f in range(num_factories):
    for p in range(num_products):
        solver.Add(solver.Sum(shipping[f, w, p] for w in range(num_warehouses)) <= production_capacity[f, p])

# Demand satisfaction constraint
for w in range(num_warehouses):
    for p in range(num_products):
        solver.Add(solver.Sum(shipping[f, w, p] for f in range(num_factories)) >= demand[w, p])

# Solve
status = solver.Solve()

if status == pywraplp.Solver.OPTIMAL:
    print('Solution:')
    print('Objective value =', solver.Objective().Value())
    for f in range(num_factories):
        for w in range(num_warehouses):
            for p in range(num_products):
                if shipping[f, w, p].solution_value() > 0:
                    print(f'Shipping from Factory {f} to Warehouse {w} for Product {p}: {shipping[f, w, p].solution_value()} units')
else:
    print('The problem does not have an optimal solution.')


Solution:
Objective value = 8471560.849391686
Shipping from Factory 0 to Warehouse 1 for Product 45: 1008.0 units
Shipping from Factory 0 to Warehouse 3 for Product 35: 194.0 units
Shipping from Factory 0 to Warehouse 5 for Product 32: 554.0 units
Shipping from Factory 0 to Warehouse 6 for Product 33: 164.0 units
Shipping from Factory 0 to Warehouse 8 for Product 0: 278.0 units
Shipping from Factory 0 to Warehouse 9 for Product 40: 491.0 units
Shipping from Factory 0 to Warehouse 11 for Product 13: 161.0 units
Shipping from Factory 0 to Warehouse 12 for Product 25: 1129.0 units
Shipping from Factory 0 to Warehouse 13 for Product 14: 577.0 units
Shipping from Factory 0 to Warehouse 13 for Product 16: 114.0 units
Shipping from Factory 0 to Warehouse 15 for Product 20: 84.0 units
Shipping from Factory 0 to Warehouse 18 for Product 32: 524.0 units
Shipping from Factory 0 to Warehouse 19 for Product 19: 96.0 units
Shipping from Factory 0 to Warehouse 22 for Product 20: 533.0 units
Shipping 